### Model Comparison: Custom Model vs. Pre-trained Model on Real Images

This notebook compares the performance of our custom model against a pre-trained model using real-world image data.

#### Results:
- **Our Model Accuracy:** 80%  
- **Pre-trained Model Accuracy:** 87%

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from skimage import io
import pandas as pd
from sklearn.utils import shuffle
import pickle

#models
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
#pca
from sklearn.decomposition import PCA

#preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# from transformers import TrOCRProcessor, VisionEncoderDecoderModel
# from PIL import Image
# import torch

In [1]:
IMG_SIZE=32

In [30]:
with open('knn_hog.pkl', 'rb') as f:
    my_model=pickle.load(f)

In [6]:

def predict_number_from_pretrained(image):
    # Open the image
    ##gray to rgb
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    image = Image.fromarray(image)
    
    processor = TrOCRProcessor.from_pretrained("microsoft/trocr-large-printed")
    pretrained_model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-large-printed")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    pixel_values = processor(image, return_tensors="pt").pixel_values

    # Ensure input is on the same device as the model
    pixel_values = pixel_values.to(device)
    pretrained_model.to(device)

    # Generate predictions
    generated_ids = pretrained_model.generate(pixel_values)
    predicted_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return predicted_text

In [32]:
def extract_hog_features(img):
    """
    Extracts Histogram of Oriented Gradients (HOG) features from an image.

    Parameters:
    img (numpy.ndarray): A numpy array of shape (H, W) representing the input image.

    Returns:
    numpy.ndarray: A numpy array of shape (N,) containing the HOG features.
    """
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    win_size = (IMG_SIZE, IMG_SIZE)
    cell_size = (4, 4)
    block_size_in_cells = (2, 2)
    
    block_size = (block_size_in_cells[1] * cell_size[1], block_size_in_cells[0] * cell_size[0])
    block_stride = (cell_size[1], cell_size[0])
    nbins = 9 
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    h = hog.compute(img)
    h = h.flatten()
    return h.flatten()


def predict_my_model(img,my_model):
    features=extract_hog_features(img).reshape((1,-1))
    return my_model.predict(features)


In [14]:
def getacc(path_to_dataset, my_model, device, processor, pretrained_model, pretrained=False):
    labels_folders = os.listdir(path_to_dataset)
    total_counter = 0
    correct_counter = 0
    
    for label in labels_folders:
        label_path = os.path.join(path_to_dataset, label)
        img_filenames = os.listdir(label_path)
        for fn in img_filenames:
            total_counter += 1
            img_path = os.path.join(label_path, fn)
            img_gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img_gray = cv2.resize(img_gray, (32, 32))
            
            if pretrained:
                result = predict_number_from_pretrained(img_gray)
            else:
                result = predict_my_model(img_gray, my_model)
            
            # Check if result is not an alphabet
            if not str(result).isalpha():
                # Assuming labels are numeric, cast label to an integer
                if int(result) == int(label):
                    correct_counter += 1
    
    return correct_counter / total_counter if total_counter > 0 else 0


In [9]:
import warnings
from sklearn.exceptions import DataConversionWarning

# Suppress specific sklearn warnings
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

In [ ]:
device=None
processor=None
pretrained_model=None
print(my_model)
print(getacc('test', my_model, device, processor, pretrained_model, pretrained=False))
# print(getacc('/kaggle/input/test-acc-dataset/test_data', my_model, device, processor, pretrained_model, pretrained=True))